In [35]:
import pandas as pd
import numpy as np
import re 
import requests 
import matplotlib 
import seaborn as sns
from bs4 import BeautifulSoup 
from pandasql import sqldf
%matplotlib inline
import sqlite3
from fractions import Fraction

In [36]:
#CSV files
dfcsv_1 = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
dfcsv_2 = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
dfcsv_3 = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
dfcsv_4 = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
dfcsv_5 = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
dfcsv_6 = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
dfcsv_7 = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
dfcsv_8 = pd.read_csv('zippedData/tmdb.movies.csv.gz')

In [37]:
#TSV FILES
dftsv_1= pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t',encoding='utf-8')
dftsv_2 = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t',encoding='unicode_escape')


In [38]:
#dfcsv_1 Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')
#dfcsv_2 Index(['nconst', 'primary_name', 'birth_year', 'death_year', 'primary_profession', 'known_for_titles'], dtype='object')
#dfcsv_3 Index(['title_id', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'is_original_title'], dtype='object')
#dfcsv_4 Index(['tconst', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres'], dtype='object')
#dfcsv_5 Index(['tconst', 'directors', 'writers'], dtype='object')
#dfcsv_6 Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
#dfcsv_7 Index(['tconst', 'averagerating', 'numvotes'], dtype='object')
#dfcsv_1 Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime', 'studio'], dtype='object')
#dftsv_2 Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher', 'date'], dtype='object')

In [46]:
dftsv_1['rating'].rename('parent_rating')

#keeping non-numeric columns with few NaN values so we could still use 
#to groupby or search from, can always drop later subbing for unknown 
#so we don't get and NaN errors 

clean1 = dfcsv_1.drop(labels='foreign_gross', axis=1).fillna(dfcsv_1['domestic_gross'].median())
clean2 = dfcsv_2.drop(labels='birth_year', axis=1).drop(labels='death_year', axis=1).fillna('unknown')
clean3 = dfcsv_3.drop(labels='attributes', axis=1).drop(labels='language', axis=1).drop(labels='types', axis=1).fillna('unknown')
clean4 = dfcsv_4.fillna(dfcsv_4['runtime_minutes'].median()).fillna('unknown')
clean5 = dfcsv_5.fillna('unknown')
clean6 = dfcsv_6.drop(labels='job', axis=1).drop(labels='characters', axis=1)
clean7 = dfcsv_7
clean8 = dfcsv_8.drop(labels='Unnamed: 0', axis=1).drop(labels='popularity', axis=1)
#columns full but useless 
clean9 = dftsv_2.drop(labels='rating', axis=1).fillna('unknown')
#rating probably not worth trying to add, would have to convert a lot to determine mean/med/mode
#review subbed for unknown so we could still use to search 
clean10 = dftsv_1 #Really short but lots of data, join on left & right in sep DF w/clean1 

In [56]:
#c1 = clean1.groupby(['studio', 'title']).mean()
#c1
#dftsv_2['rating_percent'] = [for rating in dftsv_2(): ]
print(dfcsv_8.isna().sum())
dfcsv_8['vote_count'].describe()

#float(sum(Fraction(s) for s in clean9['rating']))

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64


count    26517.000000
mean       194.224837
std        960.961095
min          1.000000
25%          2.000000
50%          5.000000
75%         28.000000
max      22186.000000
Name: vote_count, dtype: float64

In [52]:
#print(dfcsv_5.median())
#print(dfcsv_5.notna().sum())
print(dftsv_3.isna().sum())


Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64


In [11]:
cleanup1 = clean1.merge(clean3, on='title').fillna('unknown').groupby(['title'])
cleanup1

0.75